In [0]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
from google.colab import drive # remove later
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Reshape, Conv2D, MaxPooling2D 
from tensorflow.keras.layers import CuDNNLSTM, Dropout, BatchNormalization
%matplotlib inline

### ------------------------------------------------------------------------ ###
# Create time stamp
start_time = datetime.now()
print("="*80)
print("Script Execution initilize at", start_time)
print("="*80,"\n\n")
### ------------------------------------------------------------------------ ###

# Mounting Google Drive, remove later
drive.mount("/content/drive")

# Load dataset
feat = np.load("/content/drive/My Drive/ML Group Project Folder/src/feat.npy", allow_pickle = True)
path = np.load("/content/drive/My Drive/ML Group Project Folder/src/path.npy")
train = pd.read_csv("/content/drive/My Drive/ML Group Project Folder/src/train.csv")
test = pd.read_csv("/content/drive/My Drive/ML Group Project Folder/src/test.csv")

# Merge MFCC footprint with Path
df = pd.DataFrame({'path': path, 'MFCC': feat})

# Tag path in train and test dataset with MFCC Footprint
train_container = train.merge(df, on=['path'])
test_container = test.merge(df, on=['path'])

# Remove column Path:
# Now, each dataset would only contain MFCC Footprint and word label
train_set = train_container.drop(columns=["path"])
test_set = test_container.drop(columns=["path"])

# Convert features and corresponding classification labels into numpy arrays
X = np.array(train_set.MFCC.tolist())
y = np.array(train_set.word.tolist())

# Encode the classification labels
le = LabelEncoder()
class_label = le.fit_transform(y)

# Transform the data
n, t, c = X.shape[0], X[0].shape[0], X[0][0].shape[0]
container_X = np.zeros(shape=(n, t, c))
for i in range(n):
 for j in range(t):
   for k in range(c):
     try:
       container_X[i, j, k] = X[i][j][k]
     except:
        pass

X_train, X_val, y_train, y_val = train_test_split(container_X, class_label,
                                                  test_size=0.2,
                                                  random_state = 24,
                                                  stratify = class_label)
def grid_search(batch, lr):

    model = Sequential()

    model.add(BatchNormalization(input_shape=(X_train.shape[1:])))
    model.add(Reshape((99,13,1)))

    model.add(Conv2D(filters=256, kernel_size=(4,4), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(filters=64, kernel_size=(2,2), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Conv2D(filters=32, kernel_size=(2,2), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Reshape((44,32)))
    model.add(CuDNNLSTM(256))
    model.add(Dropout(0.1))
    model.add(Dense(35, activation = "softmax"))

    # Compile the model
    opt = tf.keras.optimizers.Adam(learning_rate= lr, decay=1e-5)
    model.compile(loss = "sparse_categorical_crossentropy",
                  optimizer = opt,
                  metrics = ["sparse_categorical_accuracy"])

    # Display model architecture summary 
    model.summary()

    # Calculate pre-training accuracy 
    score = model.evaluate(X_val, y_val, verbose=1)
    accuracy = 100*score[1]

    print("Pre-training accuracy: %.4f%%" % accuracy)
    print("\n\n")

    ### ------------------------------------------------------------------------ ###
    # Create time stamp
    cp1 = datetime.now() - start_time
    print("="*80)
    print("Time elapsed:", cp1)
    print("Training is about to begin")
    print("="*80,"\n\n")
    cp1 = datetime.now()
    ### ------------------------------------------------------------------------ ###


    ### ------------------------------------------------------------------------ ###
    ### ------------------------   Training the Model   ------------------------ ###
    ### ------------------------------------------------------------------------ ###

    # Define Callbacks
    cb = [ModelCheckpoint(
        filepath="/content/drive/My Drive/out/Conv2D LSTM Stratify Model Kwastas 1 CP.hdf5",
        verbose=1, save_best_only=True), #change path
          EarlyStopping(monitor = "val_loss", patience = 10, 
                        mode = "min", verbose = 1)]

    # For logging exec time
    start = datetime.now()

    # Fit model
    model_result = model.fit(X_train, y_train, epochs = 1000, batch_size = batch,
                             validation_data=(X_val, y_val), 
                             callbacks=cb, verbose=1)

    # For displaying time
    duration = datetime.now() - start
    print("Training completed in time: ", duration)

    # Save current model
    model.save("/content/drive/My Drive/out/Conv2D LSTM Stratify Model Kwastas 1.h5")
    print("Model is saved in /content/drive/My Drive/out/Conv2D LSTM Stratify Model Kwastas 1.h5") # change later

    # Model Evaluation
    # Load the best weight model
    model.load_weights("/content/drive/My Drive/out/Conv2D LSTM Stratify Model Kwastas 1 CP.hdf5")
    _, train_acc = model.evaluate(X_train, y_train, verbose=1)
    _, val_acc = model.evaluate(X_val, y_val, verbose=1)
    print("\n\n")
    print("="*80)
    print("Evaluation on the best model")
    print('Train: %.3f \nTest: %.3f' % (train_acc, val_acc))
    print("="*80, "\n")

    # Plotting model_result
    plt.plot(model_result.history['sparse_categorical_accuracy'], label='train')
    plt.plot(model_result.history['val_sparse_categorical_accuracy'], label='test')
    plt.title("Model Accuracy")
    plt.legend(["train", "test"], loc = "lower right")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.show()
    print("\n")
    print("="*80, "\n")


    plt.plot(model_result.history['loss'], label='train')
    plt.plot(model_result.history['val_loss'], label='test')
    plt.title("Model Error")
    plt.legend(["train", "test"], loc = "upper right")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.show()

    hyperparameters = (lr, batch)
    result[hyperparameters] = (val_acc)
    print (result)


learning_rate = [0.001, 0.0001]
batch_size = [32,64]
result = {}

for batch in batch_size:
  for lr in learning_rate:
    grid_search(batch, lr)